# Regression for Seasonality Analysis

We've explored using a regression for time series forecasting, but what if there are seasonal or cyclical patterns in the data?

Let's explore an example of how to use regression to identify cyclical patterns and perform seasonality analysis with time series data.


In [ ]:
#| echo: false

#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

from pandas import set_option
set_option('display.max_rows', 6)

## Data Loading


For a time series dataset that exemplifies cyclical patterns, let's consider this dataset of U.S. employment over time, from the Federal Reserve Economic Data (FRED).

Fetching the data, going back as far as possible:


In [ ]:
from pandas_datareader import get_data_fred
from datetime import datetime

DATASET_NAME = "PAYNSA"
df = get_data_fred(DATASET_NAME, start=datetime(1900,1,1))
print(len(df))
df

:::{.callout-tip title="Data Source"}
Here is some more information about the ["PAYNSA" dataset](https://fred.stlouisfed.org/series/PAYNSA):

"All Employees: Total Nonfarm, commonly known as Total Nonfarm Payroll, is a measure of the number of U.S. workers in the economy that excludes proprietors, private household employees, unpaid volunteers, farm employees, and the unincorporated self-employed."

"Generally, the U.S. labor force and levels of employment and unemployment are subject to fluctuations due to seasonal changes in weather, major holidays, and the opening and closing of schools."

"The Bureau of Labor Statistics (BLS) adjusts the data to offset the seasonal effects to show non-seasonal changes: for example, women's participation in the labor force; or a general decline in the number of employees, a possible indication of a downturn in the economy.

To closely examine seasonal and non-seasonal changes, the BLS releases two monthly statistical measures: the seasonally adjusted All Employees: Total Nonfarm (PAYEMS) and All Employees: Total Nonfarm (PAYNSA), which is not seasonally adjusted."

This "PYYNSA" data is expressed in "Thousands of Persons", and is "Not Seasonally Adjusted".

The dataset frequency is "Monthly".
:::




Wrangling the data, including renaming columns and converting the date index to be datetime-aware, may make it easier for us to work with this data:


In [ ]:
from pandas import to_datetime

df.rename(columns={DATASET_NAME: "employment"}, inplace=True)
df.index.name = "date"
df.index = to_datetime(df.index)
df

## Data Exploration


Visualizing the data:


In [ ]:
import plotly.express as px

px.line(df, y="employment", height=450,
        title="US Employment by month (non-seasonally adjusted)",
        labels={"employment": "Employment (in thousands of persons)"},
)

**Cyclical Patterns**

Exploring cyclical patterns in the data:


In [ ]:
px.line(df[(df.index.year >= 1970) & (df.index.year <= 1980)], y="employment",
        title="US Employment by month (selected years)", height=450,
        labels={"Employment": "Employment (in thousands)"},
)

:::{.callout-tip title="Interactive dataviz"}
Hover over the dataviz to see which month(s) typically have higher employment, and which month(s) typically have lower employment.
:::

**Trend Analysis**

Exploring trends:


In [ ]:
import plotly.express as px

px.scatter(df, y="employment",  height=450,
        title="US Employment by month (vs Trend)",
        labels={"employment": "Employment (in thousands)"},
        trendline="ols", trendline_color_override="red"
)

Looks like evidence of a possible linear relationship. Let's perform a more formal regression analysis.


## Data Encoding

Because we need numeric features to perform a regression, we convert the dates to a linear time step of integers (after sorting the data first for good measure):


In [ ]:
df.sort_values(by="date", ascending=True, inplace=True)

df["time_step"] = range(1, len(df) + 1)
df

We will use the numeric time step as our input variable (`x`), to predict the employment (`y`).










## Data Splitting

**X/Y Split**

Identifying dependent and independent variables:


In [ ]:
x = df[["time_step"]]

y = df["employment"]

print("X:", x.shape)
print("Y:", y.shape)

**Adding Constants**

We are going to use `statsmodels`, so we add a column of constant ones representing the intercept:


In [ ]:
import statsmodels.api as sm

# adding in a column of constants, as per the OLS docs
x = sm.add_constant(x)
x.head()

**Train/Test Split**

Splitting into training vs testing datasets:


In [ ]:
#from sklearn.model_selection import train_test_split
#
#x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=99)
#print("TRAIN:", x_train.shape, y_train.shape)
#print("TEST:", x_test.shape, y_test.shape)

Splitting data sequentially where earlier data is used in training and recent data is use for testing:


In [ ]:
#print(len(df))
#
#training_size = round(len(df) * .8)
#print(training_size)
#
#x_train = x.iloc[:training_size] # slice all before
#y_train = y.iloc[:training_size] # slice all before
#
#x_test = x.iloc[training_size:] # slice all after
#y_test = y.iloc[training_size:] # slice all after
#print("TRAIN:", x_train.shape)
#print("TEST:", x_test.shape)

For this example, we will not split the data. To help illustrate a story about predictions over the entire time period.




## Model Selection and Training

Training a linear regression model on the training data:


In [ ]:
import statsmodels.api as sm

model = sm.OLS(y, x, missing="drop")
print(type(model))

results = model.fit()
print(type(results))

Examining training results:


In [ ]:
print(results.summary())

In [ ]:
print(results.params)
print("------------")
print(f"y = {results.params['time_step'].round(3)}x + {results.params['const'].round(3)}")

In [ ]:
df["prediction"] = results.fittedvalues
df["residual"] = results.resid

In [ ]:
#from pandas import DataFrame
#
## get all rows from the original dataset that wound up in the training set:
#training_set = df.loc[x_train.index].copy()
#print(len(training_set))
#
## create a dataset for the predictions and the residuals:
#training_preds = DataFrame({
#    "prediction": results.fittedvalues,
#    "residual": results.resid
#})
## merge the training set with the results:
#training_set = training_set.merge(training_preds,
#    how="inner", left_index=True, right_index=True
#)
#
## calculate error for each datapoint:
#training_set

**Regression Trends**

Plotting trend line:


In [ ]:
px.line(df, y=["employment", "prediction"], height=350,
    title="US Employment (monthly) vs linear trend",
    labels={"value":""}
)

**Regression Residuals**

Removing the trend, plotting just the residuals:


In [ ]:
px.line(df, y="residual",
    title="US Employment (monthly) vs linear trend residuals", height=350
)

There seem to be some periodic movements in the residuals.

#### Seasonality via Means of Periodic Residuals

Observe there may be some cyclical patterns in the residuals, by calculating periodic means:


In [ ]:
#| echo: false

from pandas import set_option
set_option('display.max_rows', 15)

In [ ]:
df["year"] = df.index.year
df["quarter"] = df.index.quarter
df["month"] = df.index.month

Here we are grouping the data by quarter and calculating the average residual. This shows us for each quarter, on average, whether predictions are above or below trend:


In [ ]:
df.groupby("quarter")["residual"].mean()

In [ ]:
df.groupby("month")["residual"].mean()

In [ ]:
#| echo: false

from pandas import set_option
set_option('display.max_rows', 6)

#### Seasonality via Regression on Periodic Residuals

Let's perform a regression using months as the features and the trend residuals as the target. This can help us understand the degree to which employment will be over or under trend for a given month.


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
# "one hot encode" the monthly values:
from pandas import get_dummies as one_hot_encode

x_monthly = one_hot_encode(df["month"])
x_monthly.columns=["Jan", "Feb", "Mar", "Apr",
                "May", "Jun", "Jul", "Aug",
                "Sep", "Oct", "Nov", "Dec"]
x_monthly = x_monthly.astype(int)
x_monthly

In [ ]:
y_monthly = df["residual"]

ols_monthly = sm.OLS(y_monthly, x_monthly)
print(type(ols_monthly))

results_monthly = ols_monthly.fit()
print(type(results_monthly))

print(results_monthly.summary())

The coefficients tell us how each month contributes towards the regression residuals, in other words, for each month, to what degree does the model predict we will be above or below trend?

**Monthly Predictions of Residuals**


In [ ]:
df["prediction_monthly"] = results_monthly.fittedvalues
df["residual_monthly"] = results_monthly.resid

Decomposition of the original data into trend, seasonal component, and residuals:


In [ ]:
px.line(df, y=["employment", "prediction"], title="Employment vs trend", height=350)

In [ ]:
px.line(df, y="prediction_monthly", title="Employment seasonal component", height=350)

In [ ]:
px.line(df, y="residual_monthly", title="Employment de-trended residual", height=350)